In [1]:
import matplotlib.pyplot as plt
import random

In [2]:
import pickle

def save_data_with_pickle(relative_path, data):
    """ Save data using pickle (serialize) """

    with open(relative_path, 'wb') as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
def load_data_with_pickle(relative_path):
    """ Load data using pickle (deserialize) """

    with open(relative_path, 'rb') as handle:
        return pickle.load(handle)

### Load Dataset

In [3]:
dataset = load_data_with_pickle('/datahdd/vmanuel/datasets/word_and_composite_dataset')

### Create DBpedia Direct Graph

In [4]:
import networkx as nx

G = nx.DiGraph()

with open('/datahdd/vmanuel/datasets/dbpedia_edgelist_no_closure.tsv') as inp:
    lines = inp.readlines()
    for line in lines:
        line = line.replace('\n', '')
        couple = [c.lower() for c in line.split(' ')]
        if not G.has_node(couple[0]):
            G.add_node(couple[0])
        if not G.has_node(couple[1]):
            G.add_node(couple[1])
        G.add_edge(couple[0], couple[1])


In [8]:
reverse_dataset = {}

for k, words in dataset.items():
    for w in words:
        try:
            reverse_dataset[w].append(k)
        except:
            reverse_dataset[w] = [k]

In [9]:
i = 0
for r in reverse_dataset.values():
    if len(r) > 1:
        i += 1
        print(r)
i

0

In [7]:
for k, v in reverse_dataset.items():
    if len(v) > 1:
        min_dist = 100
        for clas in v:
            d = nx.shortest_path_length(G, 'thing', clas.lower())
            if d < min_dist:
                min_dist = d
                min_k = clas.lower()
            if d == min_dist:
                x = random.random()
                if x > 0.5:
                    min_dist = d
                    min_k = clas.lower()
        for clas in v:
            if clas != min_k:
                dataset[clas].remove(k)


In [10]:
save_data_with_pickle('/datahdd/vmanuel/datasets/minimal_type_dataset', dataset)